In [ ]:
#Before running the cells,install this 3 pacakges in our virtual env dublinbus:

# pip install cryptography
# pip install seaborn
# pip install sklearn


In [ ]:
import pymysql,json
def extract_tripsid_46A():

    sql = "select distinct trips100.TRIPID,trips100.LINEID from dublinbus.trips100 where trips100.LINEID='46A' and DAYOFSERVICE like '%DEC%'"
    db = pymysql.connect(host="127.0.0.1",  # your host
                         port = 3307,
                         user="admin",  # username
                         password="Ucd-nl-2018",  # password
                         db="dublinbus", # name of the database
                        use_unicode=True, 
                         charset='utf8') 


    cursor = db.cursor()
    cursor.execute(sql)
    result = cursor.fetchall()
    print(result)
    return result

In [ ]:
result = extract_tripsid_46A()

In [ ]:
import pymysql,json
def create_list_46A(trip_id):

    sql = " select * from dublinbus.minLeavetimes where TRIPID =%s" 
    db = pymysql.connect(host="127.0.0.1",  # your host
                         port = 3307,
                         user="admin",  # username
                         password="Ucd-nl-2018",  # password
                         db="dublinbus", # name of the database
                        use_unicode=True, 
                         charset='utf8') 


    cursor = db.cursor()
    cursor.execute(sql,(trip_id,))
    result = cursor.fetchall()
    print(result)
    return result

In [ ]:
full_list_46A = []
for i in result:
    get_data = create_list_46A(i[0])
    full_list_46A.append(get_data)

In [ ]:
len(full_list_46A)

In [ ]:
import numpy as np
import pandas as pd
# Creating an empty Dataframe with column names only
df = pd.DataFrame()
for i in full_list_46A:
    i = list(i)
    df = df.append(i, ignore_index = True)


In [ ]:
print(df.tail(186))

In [ ]:
df_46A = df.copy()

In [ ]:
df_46A.drop(df_46A.columns[[1,5,8]],axis=1,inplace=True)

In [ ]:
df_46A.head(5)

In [ ]:
date = df_46A[[0]]

In [ ]:
date = list(date[0])

In [ ]:
date[0]

In [ ]:
extract_date = []
for i in date:
    new_date = i[0:9]
    extract_date.append(new_date)
    

In [ ]:
import datetime
extract_week_day = []
for i in extract_date:
    extract_week_day.append(datetime.datetime.strptime(i, '%d-%b-%y').strftime('%A'))

In [ ]:
df_46A = df_46A.assign(Extracted_Date=extract_date)

In [ ]:
df_46A = df_46A.assign(Day=extract_week_day)

In [ ]:
df_46A.rename({0:'DateTime', 2: 'Sequence',3:'StopID',4:'Planned_Arr',6:'Actual_Arr',7:'Actual_Depart'},axis=1,inplace=True)  # new method


In [ ]:
df_46A

In [ ]:
#Creating Dummies for the day of the week
data_input1= pd.DataFrame(df_46A['Day'])

In [ ]:
dummy=pd.get_dummies(data_input1)

In [ ]:
df_46A=pd.concat([df_46A,dummy],axis=1)

In [ ]:
df_46A.head(5)

In [ ]:
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.options.mode.chained_assignment=None
%matplotlib inline

In [ ]:
input_to_model=pd.DataFrame(df_46A[['Sequence','StopID','Planned_Arr','Day_Monday','Day_Saturday','Day_Sunday','Day_Thursday','Day_Tuesday','Day_Wednesday']])
#Day_Friday has been taken out because if mon-sun all are zeroes that means friday is 1. Using dummy one hot encoding concept
#print(input_model)
output = df_46A['Actual_Arr']
X_train,X_test,Y_train,Y_test=train_test_split(input_to_model,output,test_size=0.33,random_state=42)
print (X_train.shape)
rf=RandomForestRegressor(n_estimators=100)
rf.fit(X_train,Y_train)
print(Y_test.shape)

In [ ]:
prediction=rf.predict(X_test)
print (prediction)
print(Y_test)

In [ ]:
#Model Evaluation Parameters: RMSE
print(np.sqrt(metrics.mean_squared_error(Y_test,prediction)))

In [ ]:
#MSLE
from sklearn.metrics import mean_squared_log_error
mean_squared_log_error(Y_test,prediction)

In [ ]:
import pickle
pickle.dump(rf,open('prediction_46A.pickle','wb'))

In [ ]:
#Testing with cPickle to see if its faster to unpickle files in django against pickle. Its what we are using now

import _pickle as cPickle
import gc
cPickle.dump(rf,open('prediction_46A_cPickle.pickle','wb'))